In [7]:
import scrapy
import time
from selenium import webdriver
#from scrapy.http.request import Request
#from selenium.webdriver.common.keys import Keys
import os
import json

rosters_dict = {}
chromedriver = "/Users/R1/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver

class bballref(scrapy.Spider):
    name = "bbref"
    custom_settings = {
            "DOWNLOAD_DELAY" : 2,
            "CONCURRENT_REQUESTS_PER_DOMAIN": 2
            }
  
    def __init__(self):
        self.driver = webdriver.Chrome(chromedriver)
        scrapy.Spider.__init__(self)

    def __del__(self):
        scrapy.Spider.__del__(self)

    def start_requests(self):
        url = 'http://www.basketball-reference.com/leagues/NBA_2016.html' 
        yield scrapy.Request(url = url, callback = self.team_links)
    
    def team_links(self, response):
        team_urls = response.xpath('//div[contains(@id, "all_confs_standings")]//a')
        for i in team_urls:
            name = i.xpath('./text()').extract()[0]
            print name
            url_stub = i.xpath('./@href').extract()[0] 
            url = 'http://www.basketball-reference.com'+ url_stub
            yield scrapy.Request(url = url, callback = self.get_stats, meta = {'team':name})


    def get_stats(self, response):
        team = response.request.meta['team']
        player_list = []
        # feed url to selenium
        self.driver.get(response.url)
        # maximize selenium window (indicates process has started)
        self.driver.maximize_window()
        # give time to load data
        time.sleep(3)
        # find totals table, loop through each row (player) and pull out relevant statistics
        for i in self.driver.find_elements_by_xpath('//table[@id="totals"]//tbody//tr'):
            try:
                player_name = i.find_element_by_xpath('.//td[@data-stat="player"]').text
                minutes_played =  i.find_element_by_xpath('.//td[@data-stat="mp"]').text
                games_played = i.find_element_by_xpath('.//td[@data-stat="g"]').text
                games_started = i.find_element_by_xpath('.//td[@data-stat="gs"]').text
                points = i.find_element_by_xpath('.//td[@data-stat="pts"]').text
                # create a tuple for each player, append to list
                player = (player_name, minutes_played, games_played, games_started, points)
                player_list.append(player)
                
            except:
                continue
        # create new entry in rosters dictionary for each team
        rosters_dict[team] = player_list
        # will save every iteration, could be more efficient
        f = open('rosters.txt','w')
        f.write(json.dumps(rosters_dict)
        
        return rosters_dict


SyntaxError: invalid syntax (<ipython-input-7-20d4f94c61c4>, line 70)